In [1]:
import cryo
import polars as pl
import binascii
import web3
import json
from eth_abi import decode

In [2]:
# The multical cantract address, but we also need ABI
MULTICALL3_ADDRESS = '0xcA11bde05977b3631167028862bE2a173976CA11'
MULTICALL3_ABI=json.loads('[{"inputs":[{"internalType":"bool","name":"requireSuccess","type":"bool"},{"components":[{"internalType":"address","name":"target","type":"address"},{"internalType":"bytes","name":"callData","type":"bytes"}],"internalType":"struct Multicall3.Call[]","name":"calls","type":"tuple[]"}],"name":"tryAggregate","outputs":[{"components":[{"internalType":"bool","name":"success","type":"bool"},{"internalType":"bytes","name":"returnData","type":"bytes"}],"internalType":"struct Multicall3.Result[]","name":"returnData","type":"tuple[]"}],"stateMutability":"payable","type":"function"}]')

UNIV2_ETH_USDC= '0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc'

In [31]:
# Function Signatures 4 bytes
getBlocknumber_4b = '42cbb15c'
getReserves_4b = '0902f1ac'

In [32]:
# Functions
def bytes_to_hexstr(b: any) -> str:
    if isinstance(b,list):
        return [bytes_to_hexstr(a) for a in b]
    return '0x' + b.hex()

def decode_outputdata_uniV2_price(b: bytes) -> list[float]:
    aggregated_data_uniV2 = decode(['(bool,bytes)[]'], outdata_uniV2)[0]

    ethusdc_reserves_raw = aggregated_data_uniV2[0]
    [eth_bal, usdc_bal, _] = decode(['uint112','uint112','uint32'], ethusdc_reserves_raw[1])

    eth_usdc_price = (eth_bal / usdc_bal)*1e12

    print(eth_usdc_price)
    

In [5]:
txs = cryo.collect('transactions', blocks=['-10:21881984'], rpc ='https://eth.merkle.io')

In [6]:
bytes_to_hexstr(txs['transaction_hash'].to_list())[:10]

['0x17185afe19659c8a5f93a2a547058db9c660805f625400dcc89c015f0b405088',
 '0x5fbf92a74814561ad37de87b5e2c1fd590211af35c7d87e5785790dbceab1830',
 '0x72abd91c2ac8a5cf8706d663c2629acb28db1771f8363469b516a8eb509393f8',
 '0x93c047dc4368b039206159771a5165f3223c6b85d1e951c058fa3000338098cd',
 '0x1ac8af088ae8f77d094366408676245819caf9bcbba06a490887596764b1261b',
 '0xd46fe78a7aa9113b89ca90444b32b243a71cc3dfb809afa8c7016e9904782e8b',
 '0x0fca8e8943eefbad34ffea7f28a18f27b2897ec6a75cd068fc523b68c00618bf',
 '0x74ad3f7a7fc8caff74fabc5933cbd9ecd32d97a0f039ab0c488f7990a184f294',
 '0xf82048f49848fd81d37187adb7de887f2e0cb85f3d2c04a85455c38cb98ffd90',
 '0x6050315ae98f0eb328c6fae15c2759b4335d9fbc7883e463c410d3e710eeca01']

In [7]:
# web3 instance, function from web3py
w3 = web3.Web3()
m3 = w3.eth.contract(address = MULTICALL3_ADDRESS, abi=MULTICALL3_ABI)

### Get Block Number

In [8]:
aggregate_calldata = [
    [
        MULTICALL3_ADDRESS,
        f'0x{getBlocknumber_4b}',
    ]
]
calldata = m3.encode_abi("tryAggregate", args=[True, aggregate_calldata])

In [9]:
# Args of the tryAggregate function
aggregate_calldata

[['0xcA11bde05977b3631167028862bE2a173976CA11', '0x42cbb15c']]

In [10]:
# calldata: this is the call data with info we want to get:
# 0xbce38bd7 = tryAggregate(bool,(address,bytes)[])
# aggregate_calldata = are the function arguments 
# address = Multicall_address, 
# bytes = Function Sign getblocknumber_4b = '42cbb15c'
# if you see the output of call data info all there

In [11]:
# This is the encoded data to use as calldata
# 0xbce38bd7 = tryAggregate(bool,(address,bytes)[])
# Multiclall Address + Block function signature
calldata

'0xbce38bd70000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000004000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000020000000000000000000000000ca11bde05977b3631167028862be2a173976ca110000000000000000000000000000000000000000000000000000000000000040000000000000000000000000000000000000000000000000000000000000000442cbb15c00000000000000000000000000000000000000000000000000000000'

In [12]:
cryo_kwargs = {
    'rpc': 'https://eth.merkle.io',
    'blocks': ['latest'], 
}
            
eth_call_df = cryo.collect(
    'eth_calls',
    to_address = [MULTICALL3_ADDRESS],
    call_data=[calldata],
     output_format="polars",
    **cryo_kwargs,
)

In [13]:
eth_call_df['output_data'][0]

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01Nfn'

In [14]:
outdata = eth_call_df['output_data'][0]

In [15]:
print(outdata.hex())

00000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000040000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000014e666e


In [16]:
decoded_data =decode(['(bool,bytes)[]'], outdata)[0]

In [17]:
# decoded_data[0]

In [18]:
# this is boolean and list first is true (we set in arg above)
# decoded_data[0][1]

In [17]:
# 0x14df71c= 21886748
#  we call it from block function
int(decoded_data[0][1].hex(),16)

21897042

## From UNI-V2: USDC >> ETH/USDC price

In [37]:
# web3 instance, function from web3py
# w3 = web3.Web3()
# m3 = w3.eth.contract(address = MULTICALL3_ADDRESS, abi=MULTICALL3_ABI)

aggregate_calldata_uniV2 = [
    [
        UNIV2_ETH_USDC,
        f'0x{getReserves_4b}',
    ]
]
calldata_uniV2 = m3.encode_abi("tryAggregate", args=[False, aggregate_calldata_uniV2])

In [38]:
print(calldata_uniV2)

0xbce38bd70000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000004000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000020000000000000000000000000b4e16d0168e52d35cacd2c6185b44281ec28c9dc000000000000000000000000000000000000000000000000000000000000004000000000000000000000000000000000000000000000000000000000000000040902f1ac00000000000000000000000000000000000000000000000000000000


In [39]:
cryo_kwargs = {
    'rpc': 'https://eth.merkle.io',
    'blocks': ['latest'], 
}
            
eth_call_uniV2_df = cryo.collect(
    'eth_calls',
    to_address = [MULTICALL3_ADDRESS],
    call_data=[calldata_uniV2],
     output_format="polars",
    **cryo_kwargs,
)

#### From UNI-V2: USDC >> Reserves

In [40]:
outdata_uniV2= eth_call_uniV2_df['output_data'][0]

In [41]:
#  0x will give in order 1-Hex USDC Reserve 2-Hex ETH Reserve 3-Timestamp
outdata_uniV2.hex()

'000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000400000000000000000000000000000000000000000000000000000000000000060000000000000000000000000000000000000000000000000000013843d36e9680000000000000000000000000000000000000000000001ae5193b2f899d45aee0000000000000000000000000000000000000000000000000000000067bc3cef'

In [42]:
decoded_data_uniV2 = decode(['(bool,bytes)[]'], outdata_uniV2)[0]

In [43]:
decoded_data_uniV2 

((True,
  b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x13\x84=6\xe9h\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\xaeQ\x93\xb2\xf8\x99\xd4Z\xee\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00g\xbc<\xef'),)

In [44]:
reserves_data_uniV2 = decode(['uint112','uint112','uint32'], decoded_data_uniV2[0][1])

In [45]:
reserves_data_uniV2

(21458683619688, 7937978190414558288622, 1740389615)

#### From UNI-V2: USDC >> ETH/USDC price Def

In [46]:
output_data_uniV2= eth_call_uniV2_df['output_data'][0]

In [47]:
decode_outputdata_uniV2_price(output_data_uniV2)

2703.2933456028213
